# 7.6 Operaciones numéricas con RDD

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
import numpy as np

conf = SparkConf().setMaster('local').setAppName('Mi programa')
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

dfspark = sqlContext.read.format('csv').option('header','true').option('inferSchema','true').load('base_datos_2008.csv',nrows=1e6)
dfspark = dfspark.sample(fraction = 0.001, withReplacement = False)
# Transformar la variable en una variable entera
dfspark = dfspark.withColumn('ArrDelay', dfspark['ArrDelay'].cast('integer'))
# Eliminar los datos faltantes de las variables
df2 = dfspark.na.drop(subset=['ArrDelay','DepDelay','Distance'])
df2 = df2.filter('ArrDelay is not NULL')
df2 = df2.dropDuplicates()

In [2]:
# Aproximación clásica de cómo se realiza la función 'sum' con funciones 'lambda'
lista = sc.parallelize(range(1,1000000))
# Lo que se hace aquí es ver cómo se suman todos los valores, del uno hasta el millón usando la función 'lambda'.
lista.reduce(lambda x,y:x+y)
# Una función lambda se usa tan sencillamente como: usamos el objeto ,punto, reduce, y le aplicamos el método

499999500000

In [3]:
lista.sum()

499999500000

In [4]:
from pyspark.sql.functions import mean, stddev, col
media = df2.select(mean(col('ArrDelay'))).collect()
std = df2.select(stddev(col('ArrDelay'))).collect()

In [5]:
std

[Row(stddev_samp(ArrDelay)=37.41220053597643)]

In [6]:
std[0][0] # Extraer el valor anterior

37.41220053597643

In [9]:
# Para realizar operaciones numéricas más complejas, como generación de nuevas columnas basándonos en otros valores, podemos usar la siguiente instrucción.
# Si aplicamos 'withColumn' a un dataframe el geneamos un nombre nuevo y le aplicamos algún tipo de cálculo.
# dataframe.withColumn('Nombre nuevo', cálculo).collect() # Para llamar la función
df2.withColumn('Diferencia',df2['ArrDelay']-df2['DepDelay']).collect()

[Row(Year=2008, Month=2, DayofMonth=1, DayOfWeek=5, DepTime='1913', CRSDepTime=1705, ArrTime='2030', CRSArrTime=1801, UniqueCarrier='YV', FlightNum=7347, TailNum='N519LR', ActualElapsedTime='77', CRSElapsedTime='56', AirTime='43', ArrDelay=149, DepDelay='128', Origin='IAD', Dest='MDT', Distance=94, TaxiIn='7', TaxiOut='27', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='149', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='0', Diferencia=21.0),
 Row(Year=2008, Month=2, DayofMonth=5, DayOfWeek=2, DepTime='1346', CRSDepTime=1230, ArrTime='1517', CRSArrTime=1402, UniqueCarrier='US', FlightNum=1929, TailNum='N762US', ActualElapsedTime='91', CRSElapsedTime='92', AirTime='72', ArrDelay=75, DepDelay='76', Origin='BOS', Dest='PHL', Distance=280, TaxiIn='2', TaxiOut='17', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='0', WeatherDelay='0', NASDelay='75', SecurityDelay='0', LateAircraftDelay='0', Diferencia=-1.0),
 Row(Year=2008, Month=2, Dayof

In [8]:
# Estandarización del retraso de llegada de los vuelos
df2.withColumn('Standard',(df2['ArrDelay'] - media[0][0])/std[0][0]).collect()

[Row(Year=2008, Month=2, DayofMonth=1, DayOfWeek=5, DepTime='1913', CRSDepTime=1705, ArrTime='2030', CRSArrTime=1801, UniqueCarrier='YV', FlightNum=7347, TailNum='N519LR', ActualElapsedTime='77', CRSElapsedTime='56', AirTime='43', ArrDelay=149, DepDelay='128', Origin='IAD', Dest='MDT', Distance=94, TaxiIn='7', TaxiOut='27', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='149', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='0', Standard=3.767403662298102),
 Row(Year=2008, Month=2, DayofMonth=5, DayOfWeek=2, DepTime='1346', CRSDepTime=1230, ArrTime='1517', CRSArrTime=1402, UniqueCarrier='US', FlightNum=1929, TailNum='N762US', ActualElapsedTime='91', CRSElapsedTime='92', AirTime='72', ArrDelay=75, DepDelay='76', Origin='BOS', Dest='PHL', Distance=280, TaxiIn='2', TaxiOut='17', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='0', WeatherDelay='0', NASDelay='75', SecurityDelay='0', LateAircraftDelay='0', Standard=1.789439283302541),
 Row(Yea